In [ ]:
import os
import numpy as np
import mne
%matplotlib qt
import matplotlib
import matplotlib.pyplot as plt



In [ ]:
from mne_bids import (BIDSPath,read_raw_bids)
bids_root = "Dataset/ds004147-filtered"

bids_path = BIDSPath(task="casinos",
                     datatype='eeg', suffix='eeg',
                     root=bids_root)

# win/loss waveforms plot

In [ ]:
# evoked - loading and grand averages + plotting

derivs_path = bids_root+"/derivatives/mne-bids-pipeline/"#"Dataset/ds004147-filtered-update/"

evoked_ave_all = []

# contrasts between win and loss for each task-cue combination
cond_list = [ 
    'Win LL',
    'Loss LL',
    'Win ML',
    'Loss ML',
    'Win MH',
    'Loss MH',
    'Win HH',
    'Loss HH'
]

sub_list = ["27","28","29","30","31","32","33","34","35","36","37","38"]
grand_avgs = []

# for each condition (task-cue combination), create a list of evoked responses across all subjects and then average
for i in range(0,len(cond_list)):
    for sub in sub_list:
        evoked_ave_all.append([])
        evoked_ave_all[i].append(mne.Evoked(derivs_path+"sub-"+sub+"/eeg/sub-"+sub+"_task-casinos_ave.fif",proj=False,verbose=False,condition=cond_list[i]))
    
    # for this condition, average across subjects
    grand_avgs.append(mne.grand_average(evoked_ave_all[i])) 

    # name of the condition/difference curve, so that it shows up in the plot
    grand_avgs[i].comment=cond_list[i] 

mne.viz.plot_compare_evokeds(
    grand_avgs,
    combine="mean", # average erp across channels
    picks="FCz", # if we want to see only FCz
    colors=
         {'Win LL':'r',
         'Win ML':'r',
         'Win MH':'r',
         'Win HH':'r',
         'Loss LL':'b',
         'Loss ML':'b',
         'Loss MH':'b',
         'Loss HH':'b'},
    linestyles=
         {'Win LL':'solid',
         'Win ML':'dashed',
         'Win MH':'dashdot',
         'Win HH':'dotted',
         'Loss LL':'solid',
         'Loss ML':'dashed',
         'Loss MH':'dashdot',
         'Loss HH':'dotted'},
    ylim=dict(eeg=[-5, 20]),
    title=('sub: ',sub_list),
    time_unit="ms",
)

# difference waveforms plot

In [ ]:
# evoked - loading and grand averages + plotting

derivs_path = bids_root+"/derivatives/mne-bids-pipeline/"#"Dataset/ds004147-filtered-update/"

evoked_ave_all = []

# contrasts between win and loss for each task-cue combination
cond_list = [ 
    'Win LL - Loss LL',
    'Win ML - Loss ML',
    'Win MH - Loss MH',
    'Win HH - Loss HH'
]

sub_list = ["27","28","29","30","31","32","33","34","35","36","37","38"]
grand_avgs = []

# for each condition (task-cue combination), create a list of evoked responses across all subjects and then average
for i in range(0,len(cond_list)):
    # print(i)
    for sub in sub_list:
        evoked_ave_all.append([])
        evoked_ave_all[i].append(mne.Evoked(derivs_path+"sub-"+sub+"/eeg/sub-"+sub+"_task-casinos_ave.fif",proj=False,verbose=False,condition=cond_list[i]))
    
    # for this condition, average across subjects
    grand_avgs.append(mne.grand_average(evoked_ave_all[i])) 

    # name of the condition/difference curve, so that it shows up in the plot
    grand_avgs[i].comment=cond_list[i] 

mne.viz.plot_compare_evokeds(
    grand_avgs,
    combine="mean", # average erp across channels
    picks="FCz", # if we want to see only FCz
    colors=['#EE82EE','#EE82EE','#a020f0','#a020f0'],
    linestyles=['solid','dashed','dotted','dashdot'],
    ylim=dict(eeg=[-5, 10]),
    title=('sub: ',sub_list),
    time_unit="ms",
)

In [3]:
from utils import *
%matplotlib qt

fig_conditions = plot_avg_evokeds(
    sub_list=["27","28","29","30","31","32","33","34","35","36","37","38"],
    cond_list=[ 
        'Win LL',
        'Loss LL',
        'Win ML',
        'Loss ML',
        'Win MH',
        'Loss MH',
        'Win HH',
        'Loss HH'
    ],
    colors={'Win LL':'r',
         'Win ML':'r',
         'Win MH':'r',
         'Win HH':'r',
         'Loss LL':'b',
         'Loss ML':'b',
         'Loss MH':'b',
         'Loss HH':'b'},
    linestyles={'Win LL':'solid',
         'Win ML':'dashed',
         'Win MH':'dashdot',
         'Win HH':'dotted',
         'Loss LL':'solid',
         'Loss ML':'dashed',
         'Loss MH':'dashdot',
         'Loss HH':'dotted'}
    )

fig_contrasts = plot_avg_evokeds(
    bids_root = "Dataset/ds004147-filtered",
    sub_list=["27","28","29","30","31","32","33","34","35","36","37","38"],
    cond_list=[ 
        'Win LL',
        'Loss LL',
        'Win ML',
        'Loss ML',
        'Win MH',
        'Loss MH',
        'Win HH',
        'Loss HH'
    ],
    colors=['#EE82EE','#EE82EE','#a020f0','#a020f0'],
    linestyles=['solid','dashed','dotted','dashdot']
    )

# fig_conditions = plot_avg_evokeds(
#     bids_root = "Dataset/ds004147-filtered",
#     sub_list=["28","29","30","31","32","33","34","35","36","37","38"],
#     cond_list=[ 
#         'Stimulus/S  6',
#         'Stimulus/S 16'
#     ],
#     colors={'Stimulus/S  6':'r',
#          'Stimulus/S 16':'b'
#            },
#     linestyles={'Stimulus/S  6':'solid',
#          'Stimulus/S 16':'dotted'
#                }
#     )

# fig_conditions = plot_avg_evokeds(
#     bids_root = "Dataset/ds004147-filtered",
#     sub_list=["29","30","31","32","33","34","35","36","37","38"],
#     cond_list=[ 
#         'Stimulus/S  6',
#         'Stimulus/S  7'
#     ],
#     # colors={'Stimulus/S  6':'r',
#     #      'Stimulus/S  7':'b'
#     #        },
#     # linestyles={'Stimulus/S  6':'solid',
#     #      'Stimulus/S  7':'dotted'
#     #            }
#     )

Identifying common channels ...
Identifying common channels ...
combining channels using "mean"
combining channels using "mean"


/media/mnk/Delta/AAUniStuttgart/A Sem 3/EEG/EEG2324Brownie/utils.py:504: RuntimeWarning: Only 1 channel in "picks"; cannot combine by method "mean".
  return mne.viz.plot_compare_evokeds(


TypeError: plot_avg_evokeds() missing 2 required positional arguments: 'colors' and 'linestyles'